In [1]:
import numpy as np

In [2]:
def relu(input):
    return max(0, input)

In [3]:
# Multi- Layer Neural Network

import collections


def forwardPropagation(input, weights):
    number_of_layers = len(weights['hidden'])
    activations = {}
    
    for layer in range(number_of_layers):
        hidden_layer_size = len(weights['hidden'][layer])
        hidden_layer_values = np.zeros(hidden_layer_size)
        activations[layer] = {}
        for i in range(hidden_layer_size):
            hidden_layer_values[i] = np.dot(input, weights['hidden'][layer][i])
            hidden_layer_values[i] = relu(hidden_layer_values[i])
            activations[layer][i] = hidden_layer_values[i]
        
        input = hidden_layer_values
    # print(activations)
    output_values = np.dot(input,weights['output'])
    return output_values, activations


In [4]:
def relu_derivative(x):
    # Derivative of ReLU function
    return np.where(x > 0, 1, 0)

In [5]:
def backPropagation(input, weights,target_value, learning_rate = 1):
    number_of_layers = len(weights['hidden'])
    
    predicted_value, activations = forwardPropagation(input, weights)
    
    print("Forward Propagation value: ", predicted_value)
    print("Activations", activations)
    
    delta_output_layer = np.subtract(predicted_value,target_value)
    deltas = {'output':delta_output_layer}
    
    # print("Delta output", deltas)
    #reverse track from last layer to the first
    deltas['hidden'] = {}
    for layer in range(number_of_layers - 1, -1, -1):
        # print("\n l: ", layer)
        # getting the layer weights and layer_activations
        hidden_layer_size = len(weights['hidden'][layer])
        layer_weights = weights['hidden'][layer]
        layer_activations = activations[layer]
    
        deltas['hidden'][layer] = {}
        # print("Deltas hidden", deltas )
        #computing deltas for hidden layers
        
        if layer == number_of_layers - 1:
            #layer before the output layer
            next_layer_weights = weights['output']
            next_layer_deltas = deltas['output']
            weighted_sum_next_layer = np.dot(next_layer_deltas, next_layer_weights)
        else:
            # other hidden layers
            next_layer_weights = weights['hidden'][layer + 1]
            # print("next weights", next_layer_weights)
            next_layer_deltas = deltas['hidden'][layer + 1]
            # print("next_deltas", next_layer_deltas)
            next_layer_deltas_array = np.array(list(next_layer_deltas.values()))
            # print("next_layer_deltas_array", next_layer_deltas_array)
            weighted_sum_next_layer = np.dot(next_layer_deltas_array, next_layer_weights)
            # print("weighted_sum_next_layer", weighted_sum_next_layer)
        
        for i in range(hidden_layer_size):
            # print("layer activations", layer_activations[i])
            # print("weighted_sum_next_layer", weighted_sum_next_layer[i])
            deltas['hidden'][layer][i] = np.dot(weighted_sum_next_layer[i],layer_activations[i])
    
    return deltas

In [43]:
def updateWeights(input_data, weights,target_value, learning_rate = 1):
    predicted_value, activations = forwardPropagation(input_data, weights)
    deltas = backPropagation(input_data, weights,target_value, learning_rate = 1)
    input_layer_activations = np.ones(len(input_data))


    for layer in range(len(weights['hidden'])):
        gradients = []
        # calculating gradients for hidden layer
        # print(layer)
        layer_weights = weights['hidden'][layer]
        # print("layer_weights", layer_weights)
        if layer == 0:
            layer_activations = input_layer_activations
        else:
            layer_activations = activations[layer - 1]
        # print("layer activations", layer_activations)
        next_layer_delta = deltas['hidden'][layer]
        # print("next_layer_delta ",next_layer_delta)
        
        
        for weight in range(len(layer_weights)):
            # print("\n")
            # print(layer_weights[weight])
            gradients = []
            
            for w in range(len(layer_weights[weight])):
                # print("weight", layer_weights[weight][w])
                # print("layer_activations[w]", layer_activations[w])
                # print("next_layer_delta[w] ",next_layer_delta[weight])
                gradient = layer_activations[w] * next_layer_delta[weight]
                # print("gradient", gradient)
                gradients.append(gradient)
            # print(gradients)

            layer_weights[weight] = layer_weights[weight] - (learning_rate * np.array(gradients))
            # print("new layer_weights", layer_weights[weight])
        
    #for the output layer
    output_gradient = []
    layer_activations = activations[len(weights['hidden']) - 1]
    output_delta = deltas['output']
    # print(layer_activations, "OUTPUT")
    # print("output_delta", output_delta)
    for w in range(len(weights['output'])):
        # print("layer_activations[w]", layer_activations[w])
        output_gradient.append(layer_activations[w]*output_delta)
    # print("output g", output_gradient)

    #Updating output weights:
    weights['output'] = weights['output'] - (learning_rate * np.array(output_gradient))
    
    return weights

In [49]:
input_data = np.array([2,3])
weights = {
    'hidden': {0: [np.array([0.5, 0.4]), np.array([0.3, -0.2])], 
                1: [np.array([0.6, -0.3]), np.array([0.4, 0.8])]},
    'output': np.array([1, -1])}
target_value = 1

In [50]:
for x in range(20): 
    print("\n")
    deltas = backPropagation(input_data, weights,target_value, learning_rate = 0.01)
    updated_weights = updateWeights(input_data, weights,target_value, learning_rate = 0.01)
    print("deltas", deltas)
    print("weights", weights)



Forward Propagation value:  0.43999999999999995
Activations {0: {0: 2.2, 1: 0.0}, 1: {0: 1.32, 1: 0.8800000000000001}}
Forward Propagation value:  0.43999999999999995
Activations {0: {0: 2.2, 1: 0.0}, 1: {0: 1.32, 1: 0.8800000000000001}}
deltas {'output': -0.56, 'hidden': {1: {0: -0.7392000000000001, 1: 0.4928000000000001}, 0: {0: -0.5420799999999999, 1: 0.0}}}
weights {'hidden': {0: [array([0.5054208, 0.4054208]), array([ 0.3, -0.2])], 1: [array([ 0.6162624, -0.3      ]), array([0.3891584, 0.8      ])]}, 'output': array([ 1.007392, -0.995072])}


Forward Propagation value:  0.5202006813652748
Activations {0: {0: 2.227104, 1: 0.0}, 1: {0: 1.3724804560896, 1: 0.8666962292736001}}
Forward Propagation value:  0.5202006813652748
Activations {0: {0: 2.227104, 1: 0.0}, 1: {0: 1.3724804560896, 1: 0.8666962292736001}}
deltas {'output': -0.47979931863472525, 'hidden': {1: {0: -0.663382931938533, 1: 0.4137909994661545}, 0: {0: -0.5518490100443836, 1: 0.0}}}
weights {'hidden': {0: [array([0.510